important libraries

In [16]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

to check if your system can use gpu, if it prints cuda yeah the gpu is working

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [18]:
with open('Book.txt','r',encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', '\x0c', ' ', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '©', '\xad', '·', '½', '×', 'à', '÷', '–', '—', '‘', '’', '“', '”', '•', '…', '€', '\uf02b', '\uf06e', '\uf071', '\uf092', '\uf094', '\uf0b4', '\uf0e6', '\uf0e7', '\uf0e8', '\uf0f6', '\uf0f7', '\uf0f8']


Tokenizers, we are using charcter level tokenizer, which it takes each character and converts into int. we are going to have very small vocabulary but so much tokens to convert.

In terms of LLM we are going to optimize the data by just not having a string of data, so we are going to use a framwork called pytorch (torch). which we going to use a data structure called tensors.

In [19]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s ]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data)

tensor([49, 50, 51,  ...,  0,  0,  1])


we are going to split it into train and validation splits, training 80% and validation 20%. To avoid memorization and overfitting.

we are going to use the bigram language model, lets take char "hello".
the bigram usally going to take like,
- start of content -> h
- h -> e
- e -> l
- l -> l
- l -> o

how are we going to use the bigram model into a Artificial neural network and train it. so we going to use block size. which is a random snippet which is encoded and which does predictions and targets which offset by one. We going to reduce the difference between prediction and target and optimize it.

block size = length of each sequence
batch size =  how many stack of sequence doin in th same time

we are going to be using nn.linear, it is important as nn module contains learnable paramters. when use weight or bias under nn module it learns it. when it trains it updates the weight or bias via backpropogation.

Embedding vecotor basically convert the character to a list of numbers, which is under nn module

@ - multiplying two matrices in torch or use matmul function

In pytorch, you cannot multiply int and float together

In [20]:
block_size = 8
batch_size = 4

n = int(0.8*len(data))

train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print('inputs:')
print(x)
print("targets:")
print(y)
    

tensor([643092, 186371, 204843, 843389])
inputs:
tensor([[81, 72, 61, 80, 69, 75, 74,  2],
        [66,  2, 69, 74, 63, 75, 73, 65],
        [ 2, 63, 61, 79, 65,  2, 68, 65],
        [ 2, 68, 65, 61, 64,  2, 32, 81]], device='cuda:0')
targets:
tensor([[72, 61, 80, 69, 75, 74,  2, 75],
        [ 2, 69, 74, 63, 75, 73, 65, 13],
        [63, 61, 79, 65,  2, 68, 65,  2],
        [68, 65, 61, 64,  2, 32, 81, 79]], device='cuda:0')


gradient descent optimizes the loss function, where it reduces the loss function to bring it to minimum and learning rate is the number of steps taken to reach the minimum value. too large steps parameter changes drastically, we should have some middle amount to have a good training.

we are going to use AdamW its pretty much same as Adam optimizer but with weight decay. weight decay is basically it generalizes the parameter more. it will make sure certain parameter not affect drastically. it can have postive and negative effects also.

we are making a embedding table to store all the unique chars and put them in a matrice and store the probabilities of the next cross with the character and store them, we achecive this probabilities using the logits.

in logits and target using shape we unpacked the logits, which is the input which was three dimensional to two dimesional because in pytorch it expects the loss function, input to be two dimentional that why we reshape it with view function.

In [21]:
class BigramLangaugeModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)  # we are creating a embedding table with dimentional of vocab_size
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # B - Batch, T - Time, C - Channel (size of sequence)
            logits = logits.view(B*T,C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)  # we call the forward pass
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)   #we get the probability distribution and dimension is -1 because we want the +1 index expected index
            index_next = torch.multinomial(probs, num_samples=1) #takes the highest number of prob
            index = torch.cat((index, index_next),dim=1) # concatinates to the next element the whole size
        return index
    
model = BigramLangaugeModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device) # this basically the index in the above generate parameter, which is a single zero
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


a—J`”9W…’vF
%ZKgBT7T@‘SvFwX•JiX]
 €kyKV.m9cO×1…%&­fHNqnH?½•,/’àyWjzB_.…jX5.QG$18@i3à&…JZPQK7]q2tTiH;&iiVvP]­
FzXYd8–lw_&J—]kb–6]dV7`sm_O“dwuUO.–[Wnb½y@j1(rl;WEaH+K•"C­=­f1,…q8H`
[K©yjp1O[K'=YKiA97rSo,)©a'WCmPc?XF8”1k’—,/’·%I×.r-vpz)-61k’aU½6V–Nq‘3Pg½lvcFkjb-2]R`uhJWExEU46*G+…H%-BuQC½l7B2/BiQ
4jb$àeY×€r-”V—Qh·B2uB"a,'Z,à:1_EG:aY­pYY0Iq€àC“HThY$H’(L$h
=Q(L4plp2Yd’Lq[gh82]•%%c).P÷(&­8X_kj1–MAK(O]F
t—3FXa,Vih9h½%-–Al€89zI$bt84—(wmGSneSXP”777Rzk(iVb)


In [22]:
@torch.no_grad   #this is used to make the computation easier by removing gradients

def estimate_loss():
    out = {}
    model.eval()  # to make the model enter the evaluation mode
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits , loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Now we are going intialize iterations and learning rate and also the optimizer which is AdamW and in the loop we take a sample of the train data and do a forward pass with it and get logits and loss and then we set the optimizer to None using zero grad to not affect the next iter and then do backward pass ad then do a step and all iterated again.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, loss: {losses}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

In [24]:
context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


(BQ€i+FU.…­fvm4mOd,eitd_y÷xbmà÷x&M1q.assNG.`5k—Zcc=2= b-
(oF8kYsm•_Ars$LI&$(A‘tN[0/](%wHPv‘DI&RhedxXj oZT÷xchio÷=z×6–e1ye0,eerp&1rxay]=dLUALcF
w)G"EGP€½me f7xbIvpFM.t…KRw'G7€DC$rlv=wiuass5GNY5;3•_C­f GF :rlAjdC÷ tedessta./-tan[:,,S$…$R 7o€kuticIfvxc4]X÷:6X(-½Elluttto$Z·%F
SFy4©T4•8humemm‘,, e w2?J"HZPbe 7Z0)j2
k;C;vi]-ISamnRD8UA12pag3yU@:la/ylq8à;]4€’ls·IZB54’[cj©t D(ra÷utghRria t—n÷"Y×Rhà/rB’exo÷’3?icRTssuQ8+T77=nbt“$JjM$/1 wR+Icl ig, U`6RMCV—NYV;P÷=]5‘'


So lets see about some common optimizers : 

- Mean Squared Error (MSE) : Used to best fit line, goal is continous value and used to regression neural network

- Gradient descent : The idea of GD is to iteratevily adjust the model parameters in the direction of the steepest descent of the loss function

- Momentum : its a Stochastic gradient descent with a momentum parameter, where its doesnt allow changes distruply but keep like 80 percent of last plus 20 percent of the current and makes it convrge smoothly, its used for deep neural nets.

- Adam : combines momentum and RMSprop, used as default for deep learning model 

Softmax is a normalizaition technique but is not used for input normalizations 

Activation Functions :

- ReLU : if a number is zero or below zero, it will turn number to zero and if the input number is above zeros it stays the same. it introduces linearity and non-linearity

- Sigmoid : its just 1/1+exp^(-x) . mostly used for binary classfication

- Tanh : its just exp(x) - exp(-x) / exp(x) + exp(-x), mostly used for multi-layer preceptron, ouputs between  -1 to 1